In [16]:
import os
import random

import numpy as np
# 加载飞桨相关库
import paddle
from paddle.nn import Conv2D, MaxPool2D, Linear
import paddle.nn.functional as F
# 加载visualdl
from visualdl import LogWriter

logwriter = LogWriter(logdir='./runs/mnist_experiment')

# 数据载入
class MNISTDataset():
    def __init__(self, mode='train'):
        self.mnist_data = paddle.vision.datasets.MNIST(mode=mode)

    def __getitem__(self, idx):
        data, label = self.mnist_data[idx]
        data = np.reshape(data, [1, 28, 28]).astype('float32') / 255
        label = np.reshape(label, [1]).astype('int64')
        return (data, label)

    def __len__(self):
        return len(self.mnist_data)

# 查看9张输入的训练图像的样例
# dataset = MNISTDataset(mode='train')
# image_matrix = []
# for i in range(9):
#     image, label = dataset[i]
#     # 将dataset中的CHW排列的图像转换成HWC排列再写入VisualDL
#     image_matrix.append(image.transpose([1,2,0]))


# 将九张输入图像合成长宽相同的图像网格，即3X3的图像网格
# logwriter.add_image_matrix(tag='input_images', step=1, imgs=image_matrix, rows=-1)
# tags = ['image_{}'.format(i) for i in range(9)]
# logwriter.add_embeddings('input_image_embeddings', mat=[img.reshape(-1) for img in image_matrix], metadata=tags)

train_loader = paddle.io.DataLoader(MNISTDataset(mode='train'),
                                    batch_size=16,
                                    shuffle=True)

test_loader = paddle.io.DataLoader(MNISTDataset(mode='test'),
                                   batch_size=16,
                                   shuffle=False)




# 定义mnist数据识别网络模型结构
class MNIST(paddle.nn.Layer):
    def __init__(self):
        super(MNIST, self).__init__()

        # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv1 = Conv2D(in_channels=1, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小kernel_size为2，池化步长为2
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义卷积层，输出特征通道out_channels设置为20，卷积核的大小kernel_size为5，卷积步长stride=1，padding=2
        self.conv2 = Conv2D(in_channels=20, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义池化层，池化核的大小kernel_size为2，池化步长为2
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义一层全连接层，输出维度是10
        self.fc = Linear(in_features=980, out_features=10)

    # 定义网络前向计算过程，卷积后紧接着使用池化层，最后使用全连接层计算最终输出
    # 卷积层激活函数使用Relu，全连接层激活函数使用softmax
    def forward(self, inputs):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)
        return x

#创建模型
model = MNIST()

#保存动态图模型为静态图
paddle.jit.save(model, './runs/mnist_experiment/model', [paddle.static.InputSpec([-1,1,28,28])])

#设置优化器
opt = paddle.optimizer.SGD(learning_rate=0.001, parameters=model.parameters())
EPOCH_NUM = 10

logwriter.add_hparams(hparams_dict={'lr': 0.001, 'batch_size': 16, 'opt': 'sgd'},
                      metrics_list=['train_avg_loss', 'test_avg_loss', 'test_avg_acc'])

for epoch_id in range(EPOCH_NUM):
    model.train()
    train_batchs_per_epoch = len(train_loader)
    for batch_id, data in enumerate(train_loader):
        #准备数据
        images, labels = data

        #前向计算的过程
        predicts = model(images)

        #计算损失，取一个批次样本损失的平均值
        loss = F.cross_entropy(predicts, labels)
        avg_loss = paddle.mean(loss)

        #记录当前训练Loss到VisualDL
        logwriter.add_scalar("train_avg_loss", value=avg_loss.numpy(), step=batch_id+epoch_id*(train_batchs_per_epoch))

        #记录网络中最后一个fc层的参数到VisualDL
        logwriter.add_histogram("fc_weight", values=model.fc.weight.numpy(), step=batch_id+epoch_id*(train_batchs_per_epoch))

        #每训练了100批次的数据，打印下当前Loss的情况
        if batch_id % 200 == 0:
            print("epoch: {}, batch: {}, loss is: {}".format(epoch_id, batch_id, avg_loss.numpy()))

        #后向传播，更新参数的过程
        avg_loss.backward()
        # 最小化loss,更新参数
        opt.step()
        # 清除梯度
        opt.clear_grad()

    # evaluate model after one epoch
    model.eval()
    accuracies = []
    losses = []
    class_probs = []
    class_preds = []
    for batch_id, data in enumerate(test_loader):
        #准备数据
        images, labels = data
        #前向计算的过程
        predicts = model(images)
        #计算损失
        loss = F.cross_entropy(predicts, labels)
        #计算准确率
        acc = paddle.metric.accuracy(predicts, labels)
        accuracies.append(acc.numpy())
        losses.append(loss.numpy())
        #记录用于画pr曲线需要的预测概率和类别
        class_probs_batch = [F.softmax(predict, axis=0) for predict in predicts]
        class_preds_batch = paddle.argmax(predicts, 1)

        class_probs.append(class_probs_batch)
        class_preds.append(class_preds_batch)

    test_probs = paddle.concat([paddle.stack(batch) for batch in class_probs]).numpy()
    test_preds = paddle.concat(class_preds).numpy()

    for i in range(10):
        logwriter.add_pr_curve('class_{}'.format(i), labels=(test_preds == i),predictions=test_probs[:,i], num_thresholds=100, step=epoch_id)

    avg_acc, avg_loss = np.mean(accuracies), np.mean(losses)
    print("[validation]After epoch {}: accuracy/loss: {}/{}".format(epoch_id, avg_acc, avg_loss))
    #记录当前测试集平均Loss和准确率到VisualDL
    logwriter.add_scalar("test_avg_loss", value=avg_loss, step=epoch_id)
    logwriter.add_scalar("test_avg_acc", value=avg_acc, step=epoch_id)

#保存模型参数
paddle.save(model.state_dict(), 'mnist.pdparams')

epoch: 0, batch: 0, loss is: [2.444162]
epoch: 0, batch: 200, loss is: [1.8300562]
epoch: 0, batch: 400, loss is: [1.3318727]
epoch: 0, batch: 600, loss is: [1.0272088]
epoch: 0, batch: 800, loss is: [0.57753646]
epoch: 0, batch: 1000, loss is: [0.94898736]
epoch: 0, batch: 1200, loss is: [0.55481696]
epoch: 0, batch: 1400, loss is: [0.44957757]
epoch: 0, batch: 1600, loss is: [0.24114574]
epoch: 0, batch: 1800, loss is: [0.2672563]
epoch: 0, batch: 2000, loss is: [0.53473794]
epoch: 0, batch: 2200, loss is: [0.29776996]
epoch: 0, batch: 2400, loss is: [0.18626039]
epoch: 0, batch: 2600, loss is: [0.06019416]
epoch: 0, batch: 2800, loss is: [0.19380222]
epoch: 0, batch: 3000, loss is: [0.17110974]
epoch: 0, batch: 3200, loss is: [0.25835514]
epoch: 0, batch: 3400, loss is: [0.20399052]
epoch: 0, batch: 3600, loss is: [0.5912686]


AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations